## #사용할 도구 준비

In [ ]:
# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
!pip install torchinfo
from torchinfo import summary
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = torchvision.datasets.OxfordIIITPet(root='./data', split="trainval", target_types="category", download=True, transform=transform)
testset = torchvision.datasets.OxfordIIITPet(root='./data', split="test", target_types="category", download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
image, label = trainset[0]

print(f'image shape: {image.shape}')
print(f'label: {label}')
print(f'Number of classes : {len(trainset.classes)}')

In [ ]:
train_size = torch.tensor(len(trainset))
test_size = torch.tensor(len(testset))

num_classes = len(trainset.classes)
class_names = trainset.classes


print(num_classes)
print(class_names)
print(train_size)
print(test_size)

In [ ]:
def imshow(img):
  img = img / 2 + 0.5
  npimg = img.numpy()
  return np.transpose(npimg, (1,2,0))

In [ ]:

def show_multiple_images(dataset, n_images=9):
  dataiter = iter(dataset)
  images, labels = next(dataiter)
  fig, axes = plt.subplots(3, 3, figsize=(6,6))
  axes = axes.flatten()


  for i in range(n_images):
    ax = axes[i]
    img = imshow(images[i])
    ax.imshow(img)
    ax.set_title(f"Label: {trainset.classes[labels[i]]}")
    ax.axis("off")

  plt.tight_layout()
  plt.show()
     

In [ ]:
show_multiple_images(trainloader)

In [ ]:
show_multiple_images(testloader)

### Basic Block(Resnet-34)

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, use_skip_connection=True):
        super(BasicBlock, self).__init__()
        self.use_skip_connection = use_skip_connection # 스위치 저장

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # 스위치를 이용하여 Plainnet일때는 skip connection 꺼줌
        if self.use_skip_connection:
            if self.downsample is not None:
                identity = self.downsample(x)
            out += identity

        out = self.relu(out)
        return out

### Bottleneck Block(Resnet-50)

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, use_skip_connection=True):
        super(Bottleneck, self).__init__()
        self.use_skip_connection = use_skip_connection # 스위치 저장

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        # 스위치를 이용하여 Plainnet일때는 skip connection 꺼줌
        if self.use_skip_connection:
            if self.downsample is not None:
                identity = self.downsample(x)
            out += identity

        out = self.relu(out)
        return out

## Resnet 모델 구현

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=37, use_skip_connection=True):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.use_skip_connection = use_skip_connection # 전체 모델 설정 저장

        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

  
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None


        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []

        layers.append(block(self.in_channels, out_channels, stride, downsample, use_skip_connection=self.use_skip_connection))
        self.in_channels = out_channels * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, use_skip_connection=self.use_skip_connection))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

## resnet34,50과 Plainnet모델 구현

In [ ]:
def resnet34(num_classes=37):

    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, use_skip_connection=True)

def resnet50(num_classes=37):

    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, use_skip_connection=True)

def build_plainnet(model_name, num_classes=37):
 
    if model_name == 'plain34':
        return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, use_skip_connection=False)
    elif model_name == 'plain50':
        return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, use_skip_connection=False)
    else:
        raise ValueError("Choose model_name between 'plain34' and 'plain50'")


if __name__ == "__main__":
    # 입력 이미지 (224, 224, 3) -> Batch size 포함 (1, 3, 224, 224)
    dummy_input = torch.randn(1, 3, 224, 224)
    
    # 1. ResNet-34 생성 및 확인
    model_resnet34 = resnet34()
    output_resnet = model_resnet50(dummy_input)
    print(f"[ResNet-34] Output Shape: {output_resnet.shape}")
    
    # 2. ResNet-50 생성 및 확인
    model_resnet50 = resnet50()
    output_resnet = model_resnet50(dummy_input)
    print(f"[ResNet-50] Output Shape: {output_resnet.shape}") # (1, 37)

    # 3. PlainNet-50 생성 및 확인
    model_plain50 = build_plainnet('plain50')
    output_plain = model_plain50(dummy_input)
    print(f"[PlainNet-50] Output Shape: {output_plain.shape}") # (1, 37)

    # 4. PlainNet-34 생성 및 확인
    model_plain34 = build_plainnet('plain34')
    output_plain34 = model_plain34(dummy_input)
    print(f"[PlainNet-34] Output Shape: {output_plain34.shape}") # (1, 37)

    print("\n모델완성!")
     

In [ ]:
ResNet_34 = resnet34(num_classes=1000)
summary(ResNet_34, input_size=(1, 3, 224, 224))

In [ ]:
ResNet_50 = resnet50(num_classes=1000)
summary(ResNet_50, input_size=(1, 3, 224, 224))

In [ ]:

EPOCH = 15
BATCH_SIZE = 256

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim


current_time = time.time()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_resnet34 = resnet34(num_classes=37).to(device)


for param in model_resnet34.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet34.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

resnet34_train_losses = []
resnet34_val_accuracies = []

print("ResNet-34 학습 시작...")

for epoch in range(EPOCH):
    # --- 학습 (Train) ---
    model_resnet34.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0



    resnet34_epoch_loss = loss.item()
    resnet34_train_acc = 100 * correct / total


    resnet34_train_losses.append(resnet34_epoch_loss)

    print(f"Epoch [{epoch+1}/{EPOCH}] Train Loss: {resnet34_epoch_loss:.4f}, Train Acc: {resnet34_train_acc:.2f}%")


    model_resnet34.eval() 
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnet34(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    resnet34_val_accuracies.append(val_acc)

    print(f"Epoch [{epoch + 1}/{EPOCH}] Validation Acc: {val_acc:.2f}%")

print(f"Total Training time: {time.time() - current_time:.2f} seconds")

In [ ]:
model_plain34 = build_plainnet('plain34', num_classes=37).to(device)


optimizer = optim.SGD(model_plain34.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

plain34_train_losses = []
plain34_val_accuracies = []

for epoch in range(EPOCH):

    model_plain34.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_plain34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    epoch_loss = running_loss / len(trainloader)
    train_acc = 100 * correct / total

    plain34_train_losses.append(epoch_loss)

    print(f"[PlainNet] Epoch [{epoch+1}/{EPOCH}] Loss: {epoch_loss:.4f}, Acc: {train_acc:.2f}%")


    model_plain34.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_plain34(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    plain34_val_accuracies.append(val_acc)

    print(f"[PlainNet] Epoch [{epoch+1}/{EPOCH}] Val Acc: {val_acc:.2f}%")

print("PlainNet 학습 완료!")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# 1. 학습 Loss 비교 (낮을수록 좋음)
plt.subplot(1, 2, 1)
plt.plot(resnet34_train_losses, label='ResNet-34', color='blue')
plt.plot(plain34_train_losses, label='PlainNet-34', color='red', linestyle='--')
plt.title('Model Training Loss Comparison')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)



In [ ]:
plt.plot(resnet34_val_accuracies, label='ResNet-34', color='blue')
plt.plot(plain34_val_accuracies, label='PlainNet-34', color='red', linestyle='--')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:

model_resnet50 = resnet50(num_classes=37).to(device)

optimizer = optim.SGD(model_resnet50.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


resnet50_train_losses = []
resnet50_val_accuracies = []

for epoch in range(EPOCH):
   
    model_resnet50.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    
    epoch_loss = running_loss / len(trainloader)
    resnet50_train_losses.append(epoch_loss)

    train_acc = 100 * correct / total
    print(f"[ResNet-50] Epoch [{epoch+1}/{EPOCH}] Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%")

    
    model_resnet50.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnet50(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    resnet50_val_accuracies.append(val_acc)
    print(f"            Validation Acc: {val_acc:.2f}%")

print("ResNet-50 학습 완료!")

In [ ]:



model_plain50 = build_plainnet('plain50', num_classes=37).to(device)


optimizer = optim.SGD(model_plain50.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


plain50_train_losses = []
plain50_val_accuracies = []

for epoch in range(EPOCH):
    
    model_plain50.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_plain50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    
    epoch_loss = running_loss / len(trainloader)
    plain50_train_losses.append(epoch_loss)

    train_acc = 100 * correct / total
    print(f"[PlainNet-50] Epoch [{epoch+1}/{EPOCH}] Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%")

    
    model_plain50.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_plain50(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    plain50_val_accuracies.append(val_acc)
    print(f"              Validation Acc: {val_acc:.2f}%")

print("PlainNet-50 학습 완료!")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))


plt.subplot(1, 2, 1)
plt.plot(resnet50_train_losses, label='ResNet-50', color='blue')
plt.plot(plain50_train_losses, label='PlainNet-50', color='red', linestyle='--')
plt.title('Training Loss Comparison (50 Layers)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)



In [ ]:
plt.plot(resnet50_val_accuracies, label='ResNet-50', color='blue')
plt.plot(plain50_val_accuracies, label='PlainNet-50', color='red', linestyle='--')
plt.title('Validation Accuracy Comparison (50 Layers)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
     